In [ ]:
def get_trials_for_N_cells_df(curr_ncells, gdf, NEURALDATA):
    # Get current global RIDs
    ncells_t = gdf.shape[0]
    roi_ids = np.array(gdf['roi'].values.copy())

    # Random sample w/ replacement
    rand_ixs = np.array([random.randint(0, ncells_t-1) for _ in np.arange(0, curr_ncells)])
    curr_roi_list = roi_ids[rand_ixs]
    curr_roidf = gdf[gdf['roi'].isin(curr_roi_list)].copy()

    # Make sure equal num trials per condition for all dsets
    curr_dkeys = curr_roidf['datakey'].unique()
    currd = NEURALDATA[NEURALDATA['datakey'].isin(curr_dkeys)].copy()
    min_ntrials_by_config = currd[['datakey', 'config', 'trial']].drop_duplicates().groupby(['datakey'])['config'].value_counts().min()
    #print(min_ntrials_by_config)

    d_list=[]
    for datakey, dkey_rois in curr_roidf.groupby(['datakey']):
        assert datakey in currd['datakey'].unique(), "ERROR: %s not found" % datakey
        # Get current trials, make equal to min_ntrials_by_config
        tmpd = pd.concat([trialmat.sample(n=min_ntrials_by_config)
                         for (rid, cfg), trialmat in currd[currd['datakey']==datakey].groupby(['cell', 'config'])], axis=0)
        tmpd['cell'] = tmpd['cell'].astype(float)

        # For each RID sample belonging to current dataset, get RID order
        sampled_cells = pd.concat([dkey_rois[dkey_rois['roi']==globalid][['roi', 'dset_roi']]
                                   for globalid in curr_roi_list])
        sampled_dset_rois = sampled_cells['dset_roi'].values
        sampled_global_rois = sampled_cells['roi'].values
        cell_lut = dict((k, v) for k, v in zip(sampled_dset_rois, sampled_global_rois))

        # Get response + config, replace dset roi  name with global roi name
        slist = [tmpd[tmpd['cell']==rid][['config', 'response']].rename(columns={'response': cell_lut[rid]})
                 .sort_values(by='config').reset_index(drop=True) for rid in sampled_dset_rois]
        curr_roidata = pd.concat(slist, axis=1)
        curr_roidata = curr_roidata.loc[:,~curr_roidata.T.duplicated(keep='first')]
        d_list.append(curr_roidata)
    curr_neuraldf = pd.concat(d_list, axis=1)[curr_roi_list]

    cfg_df = pd.concat(d_list, axis=1)['config']

    assert cfg_df.shape[0]==curr_neuraldf.shape[0], "Bad trials"
    if len(cfg_df.shape) > 1:
        cfg_df = cfg_df.loc[:,~cfg_df.T.duplicated(keep='first')]
        assert cfg_df.shape[1]==1, "Bad configs: %s" % str(curr_roidf['datakey'].unique()) #cfg_df

    df = pd.concat([curr_neuraldf, cfg_df], axis=1)

    return df